In [125]:
import pandas as pd
import numpy as np
import pickle

In [126]:
# Importing more complete IMDB csv file.
imdb_df = pd.read_csv('zippedData/imdb.movies.csv.zip')

# Dropping columns that won't be used.
imdb_df.drop(columns=['original_title', 'year', 'language', 'actors', 'description', 'avg_vote',\
                      'votes', 'production_company'], inplace=True)

# Renaming and simplifying column names to be more descriptive.
new_column_list = ['imdb_id', 'title', 'release_date', 'genre', 'duration', 'country', 'director',\
                   'writer', 'budget', 'domestic_income', 'worldwide_income', 'metascore',\
                   'imdb_user_score', 'imdb_critic_score']
imdb_df.columns = new_column_list

In [127]:
# Convert release date to datetime object. If date only contains year, defaulting to January 1st.
imdb_df.release_date = pd.to_datetime(imdb_df.release_date)

# Filter out movies without a release date and made before 2000.
imdb_df.drop(imdb_df[imdb_df['release_date'].dt.year < 2000].index, inplace=True)

# Filter out movies made outside the USA.
imdb_df.drop(imdb_df[imdb_df['country'] != 'USA'].index, inplace=True)

In [128]:
# Drop country, and writer columns.
imdb_df.drop(columns=['country', 'writer'], inplace=True)

# Filter out null values for budget and domestic_income.
imdb_df.dropna(subset=['budget'], inplace=True)
imdb_df.dropna(subset=['domestic_income'], inplace=True)

In [129]:
# Drop critic ratings due to assumption that it is not valuable.
imdb_df = imdb_df.drop(columns=['metascore', 'imdb_critic_score'])

# Filter out 5 entries without an imdb user score.
imdb_df.dropna(subset=['imdb_user_score'], inplace=True)

In [130]:
# Import metacritic userscore to combine with imdb user score.
metacritic_df = pd.read_csv('zippedData/metacritic_movies.csv')
# Drop unused columns from metacritic file
metacritic_df.drop(columns=['release_date', 'genre', 'meta_mixed', 'meta_negative', 'meta_positive', 'metascore',\
                           'user_mixed', 'user_negative', 'user_positive'], inplace=True)

# Rename columns to align with imdb_df.
metacritic_df.columns = ['title', 'metacritic_user_score']

In [131]:
# Merge metacritic userscore into movies_df based on titles
movies_df = pd.merge(imdb_df, metacritic_df, left_on='title', right_on='title', how='left')

In [132]:
# Drop duplicate entries based on title
movies_df.drop_duplicates(subset='title', inplace=True)

In [133]:
# Replace 'tbd' metacritic user scores with 'NaN'
movies_df['metacritic_user_score'] = movies_df['metacritic_user_score'].replace('tbd', 'NaN', regex=True)

# Convert series dtype to float.
movies_df.metacritic_user_score = movies_df.metacritic_user_score.astype(float)

In [134]:
# Replace null values with mean score.
meta_mean = movies_df.metacritic_user_score.mean()
movies_df.metacritic_user_score.fillna(value=meta_mean, inplace=True)

In [135]:
# Format budget, domestic, and worldwide boxoffice columns and delete dollar sign and extra space.
movies_df['budget'] = movies_df['budget'].astype(str)
movies_df['budget'] = movies_df['budget'].map(lambda x: x.split(' ')[1])

movies_df['domestic_income'] = movies_df['domestic_income'].astype(str)
movies_df['domestic_income'] = movies_df['domestic_income'].map(lambda x: x.split(' ')[1])

movies_df['worldwide_income'] = movies_df['worldwide_income'].astype(str)
movies_df['worldwide_income'] = movies_df['worldwide_income'].map(lambda x: x.split(' ')[1])

In [136]:
# Convert budget, domestic, and worldwide columns into floats.
movies_df['budget'] = movies_df['budget'].astype(float)
movies_df['domestic_income'] = movies_df['domestic_income'].astype(float)
movies_df['worldwide_income'] = movies_df['worldwide_income'].astype(float)

In [137]:
# Split genres in genre column into list of strings, then create columns with dummey variables for each genre.
unique_genres = set([x for genre in [genres.split(', ') for genres in movies_df['genre']] for x in genre])

def genre_binary(genre, list_of_genres):
    value = None
    if genre in list_of_genres:
        value = 1
    else:
        value = 0
    return value

for genre in unique_genres:
    movies_df[genre] = movies_df['genre'].map(lambda x: genre_binary(genre, x))

In [138]:
# Create ROI and profit/loss columns for both domestic & worldwide boxoffice.
movies_df['domestic_roi'] = (movies_df['domestic_income']/movies_df['budget'])*100
movies_df['worldwide_roi'] = (movies_df['worldwide_income']/movies_df['budget'])*100

movies_df['domestic_profit_loss'] = movies_df['domestic_income']-movies_df['budget']
movies_df['worldwide_profit_loss'] = movies_df['worldwide_income']-movies_df['budget']

In [139]:
# Convert columns with dollar figures into Millions
columns = ['budget', 'domestic_income', 'worldwide_income',\
           'domestic_profit_loss', 'worldwide_profit_loss']
for column in columns:
    movies_df[column] = movies_df[column].map(lambda x: round(x/1000000, ndigits=1))

In [140]:
# Convert ROI calculations into integers and round them.
movies_df['domestic_roi'] = movies_df['domestic_roi'].map(lambda x: int(round(x,ndigits=0)))
movies_df['worldwide_roi'] = movies_df['worldwide_roi'].map(lambda x: int(round(x,ndigits=0)))
movies_df

,imdb_id,title,release_date,genre,duration,director,budget,domestic_income,worldwide_income,imdb_user_score,...,Mystery,Animation,Drama,Western,War,History,domestic_roi,worldwide_roi,domestic_profit_loss,worldwide_profit_loss
0,tt0035423,Kate & Leopold,2002-04-05,"Comedy, Fantasy, Romance",118,James Mangold,48.0,47.1,76.0,330.0,...,0,0,0,0,0,0,98,158,-0.9,28.0
1,tt0083907,The Evil Dead,2018-10-31,Horror,85,Sam Raimi,0.3,2.4,2.9,802.0,...,0,0,0,0,0,0,686,827,2.0,2.5
2,tt0113026,The Fantasticks,2000-09-22,"Musical, Romance",86,Michael Ritchie,10.0,0.0,0.0,70.0,...,0,0,0,0,0,0,0,0,-10.0,-10.0
3,tt0118589,Glitter,2001-09-21,"Drama, Music, Romance",104,Vondie Curtis-Hall,22.0,4.3,5.3,314.0,...,0,0,1,0,0,0,19,24,-17.7,-16.7
4,tt0120188,Three Kings,2000-04-21,"Action, Adventure, Comedy",114,David O. Russell,75.0,60.7,107.8,563.0,...,0,0,0,0,0,0,81,144,-14.3,32.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,tt8186318,Overcomer,2019-08-23,"Drama, Sport",119,Alex Kendrick,5.0,34.7,35.7,108.0,...,0,0,1,0,0,0,694,714,29.7,30.7
2610,tt8350360,Annabelle Comes Home,2019-07-10,"Horror, Mystery, Thriller",106,Gary Dauberman,27.0,74.2,228.6,648.0,...,1,0,0,0,0,0,275,846,47.2,201.6
2611,tt9024106,Unplanned,2019-10-04,"Biography, Drama",109,"Chuck Konzelman, Cary Solomon",6.0,19.0,19.3,531.0,...,0,0,1,0,0,0,317,322,13.0,13.3
2612,tt9134216,Playing with Fire,2019-11-08,"Comedy, Family",96,Andy Fickman,29.9,18.8,21.8,49.0,...,0,0,0,0,0,0,63,73,-11.1,-8.1


In [141]:
movies_df.to_pickle('./movies_df.pkl')